In [70]:
from pyedflib import highlevel
from portiloop.src.demo.utils import xdf2array
import numpy as np

filename_edf = '/home/ubuntu/portiloop-software/BSP_L22_Portiloop_EDF.edf'
filename_xdf = '/home/ubuntu/portiloop-software/BSP_L22_Portiloop_XDF.xdf'

In [98]:
edf_read = highlevel.read_edf(filename_edf)
signal_edf = edf_read[0][1, :]
signal_edf.shape

(1147000,)

In [99]:
data_whole, columns = xdf2array(filename_xdf, 2)

In [100]:
signal_xdf = data_whole[:, columns.index("online_filtered_signal_portiloop")]
signal_xdf.shape

(1142166,)

In [101]:
len(signal_edf) - len(signal_xdf)

4834

In [102]:
np.isin(signal_edf, signal_xdf).sum()

0

In [103]:
print(f"{signal_edf[10000]} != {signal_xdf[10000]}")

-0.020828564888990615 != -0.3275071084499359


In [104]:
np.where(signal_edf == signal_xdf[100000])

(array([], dtype=int64),)

In [105]:
# signal_xdf = np.concatenate([signal_xdf, np.zeros(len(signal_edf) - len(signal_xdf))])

In [106]:
# signals = np.concatenate((np.expand_dims(signal_edf, 0), np.expand_dims(signal_xdf, 0)), axis = 0)

In [107]:
# signals.shape

(2, 1147000)

In [108]:
# # Create and edf file with both signals:
# channel_names = ['EDF_Data', "XDF_Data"]
# signal_headers = highlevel.make_signal_headers(channel_names, sample_frequency=250)
# headers = highlevel.make_header(patientname='L22', gender='Male')


# highlevel.write_edf('edf_file.edf', signals, signal_headers, headers)

True

In [145]:
from portiloop.src.demo.utils import OfflineSleepSpindleRealTimeStimulator
from portiloop.src.detection import SleepSpindleRealTimeDetector
from portiloop.src.processing import FilterPipeline


filter = FilterPipeline(nb_channels=1, sampling_rate=250)
detector = SleepSpindleRealTimeDetector(threshold=0.82, channel=1) # always 1 because we have only one channel
stimulator = OfflineSleepSpindleRealTimeStimulator()

In [143]:
print("Running online filtering and detection...")

points = []
online_activations = []
delayed_stims = []

# Go through the data
for index, point in enumerate(signal_xdf):
    # Filter the data
    filtered_point = filter.filter(np.array([point]))

    filtered_point = filtered_point.tolist()
    points.append(filtered_point[0])
    # Detect the spindles
    result = detector.detect([[point]])

    # if stimulation_phase != "Fast":
    #     delayed_stim = stimulation_delayer.step_timesteps(filtered_point[0])
    #     if delayed_stim:
    #         delayed_stims.append(1)
    #     else:
    #         delayed_stims.append(0)

    # Stimulate if necessary
    stim = stimulator.stimulate(result)
    if stim:
        online_activations.append(1)
    else:
        online_activations.append(0)

Running online filtering and detection...


In [144]:
len(online_activations)

1147000

In [141]:
sum(online_activations)

31